# Rotating and Cutoff of Linescan Images

### For a detailed explanation, please take a look at the example file

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import concurrent
import os

os.chdir('/home/lpa263/Competition/Datasets/train')

# List out all the csv files with the filename starting with train_ followed by numbers
fnames = pd.Series(glob.glob("train_*.csv"))

# We are going over each train csv files
for fname in fnames:
    # create the daskframe global variable
    global daskframe
    daskframe = dd.read_csv(fname)
    daskframe = daskframe.loc[:, ['x','y','linescan']]
    daskframe = daskframe.compute()
    
    # sum_zeros function rotate the picture with a given angle
    # and returns the number of pixels that are outside of rectangular boundary setup by the maximum and minimum
    # longitude and latitude of pixels whose linescan value is greater than 0 
    def sum_zeros(angle):
        rot = np.array([[np.cos(angle), np.sin(angle)],[-np.sin(angle), np.cos(angle)]])
        df = daskframe.copy()
        df[['x','y']] = df[['x','y']].dot(rot).round(0).astype(int)
        df[['x','y']] = (df[['x','y']]).round(-1)
        df = df.groupby(['x','y']).agg(np.max).reset_index()
        df = df.loc[df.linescan.gt(0),:]
        x_min, y_min, x_max, y_max = df.loc[:,['x','y']].apply([np.min, np.max]).to_numpy().reshape(-1)
        x_lim = np.round((df.x.unique().shape[0]*.05), -1).astype(int)*10
        y_lim = np.round((df.y.unique().shape[0]*.05), -1).astype(int)*10
        result = df[(df.x.ge(x_max-x_lim)|df.x.le(x_min+x_lim)|df.y.ge(y_max-y_lim)|df.y.le(y_min+y_lim))&df.linescan.gt(0)].shape[0]
        return(result)
        del df
        del im
    
    # We test out 51 different angles between 0 and pi/4
    with concurrent.futures.ProcessPoolExecutor() as executor:
        pi = np.pi
        choices = [pi/4*(x)/50 for x in range(51)]
        results = [executor.submit(sum_zeros, x).result() for x in choices]
        del daskframe
        
    # Choose the one that cut-off the largest number of pixels whose linescan value is not 0
    # That angle straighten up the linscan image.
    angle = choices[results.index(max(results))]
    df = dd.read_csv(fname)
    df = df.loc[:, ['x','y','linescan']]
    df = df.compute()
    rot = np.array([[np.cos(angle), np.sin(angle)],[-np.sin(angle), np.cos(angle)]])
    df[['x','y']] = df[['x','y']].dot(rot).round(0).astype(int)
    df[['x','y']] = (df[['x','y']]).round(-1)
    df = df.groupby(['x','y']).agg(np.max).reset_index()
    x_min, y_min, x_max, y_max = df.loc[df.linescan.gt(0),['x','y']].apply([np.min, np.max]).to_numpy().reshape(-1)
    x_lim = np.round((df.x.unique().shape[0]*.05), -1).astype(int)*10
    y_lim = np.round((df.y.unique().shape[0]*.05), -1).astype(int)*10
    df = df[(df.x.le(x_max-x_lim))&(df.x.ge(x_min+x_lim))&(df.y.le(y_max-y_lim))&(df.y.ge(y_min+y_lim))]
    df[['x','y']] = (df[['x','y']].dot(np.linalg.inv(rot))+5).round(-1)-5
    df.reset_index(drop = True, inplace = True)
    df = df.groupby(['x','y']).agg(np.max).reset_index().loc[:, ['x','y']]
    df.to_csv(f'rot_{fname}', index = False)
    del df